Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
# Make environment
env = rlcard.make('kuhn-poker', config={'seed': 0})
eval_env = rlcard.make('kuhn-poker', config={'seed': 0})

In [5]:
# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 1_000
evaluate_num = 10_000
episode_num = 100_000

In [6]:
# The paths for saving the logs and learning curves
log_dir = './experiments/kuhn_poker_dqn_result/'

In [7]:
# Set a global seed
set_global_seed(0)

In [8]:
agent = DQNAgent(
    env.action_num,
    env.state_shape[0],
    memory=2_000_000,
    lr=0.1,
    batch_size=128,
    update_every=128,
    eps_start=0.12,
    target_update=300,
    hidden_neurons=[64]
)

cfr_agent = CFRAgent(eval_env, model_path='./models/kuhn_poker_cfr')
cfr_agent.load()

random_agent = RandomAgent(action_num=eval_env.action_num)

env.set_agents([agent, agent])
eval_env.set_agents([agent, cfr_agent])

In [9]:
# Init a Logger to plot the learning curve
logger = Logger(log_dir)

In [10]:
for episode in range(episode_num):
    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.train(ts)

    # Evaluate the performance. Play with CFR agents.
    if episode % evaluate_every == 0:
        print(episode / evaluate_every)
        logger.log_performance(agent.weight_updates,
                               tournament(eval_env, evaluate_num)[0])

0.0

----------------------------------------
  timestep     |  0
  reward       |  -0.252
----------------------------------------
1.0

----------------------------------------
  timestep     |  7
  reward       |  -0.1023
----------------------------------------
2.0

----------------------------------------
  timestep     |  16
  reward       |  -0.0596
----------------------------------------
3.0

----------------------------------------
  timestep     |  25
  reward       |  -0.0002
----------------------------------------
4.0

----------------------------------------
  timestep     |  34
  reward       |  0.015
----------------------------------------
5.0

----------------------------------------
  timestep     |  44
  reward       |  0.0092
----------------------------------------
6.0

----------------------------------------
  timestep     |  53
  reward       |  0.0002
----------------------------------------
7.0

----------------------------------------
  timestep     |  62
  

61.0

----------------------------------------
  timestep     |  509
  reward       |  -0.0029
----------------------------------------
62.0

----------------------------------------
  timestep     |  516
  reward       |  -0.0561
----------------------------------------
63.0

----------------------------------------
  timestep     |  524
  reward       |  -0.0102
----------------------------------------
64.0

----------------------------------------
  timestep     |  532
  reward       |  -0.0056
----------------------------------------
65.0

----------------------------------------
  timestep     |  540
  reward       |  -0.0174
----------------------------------------
66.0

----------------------------------------
  timestep     |  548
  reward       |  0.0016
----------------------------------------
67.0

----------------------------------------
  timestep     |  556
  reward       |  0.0071
----------------------------------------
68.0

----------------------------------------
  t

In [11]:
# Close files in the logger
logger.close_files()

In [12]:
# Plot the learning curve
logger.plot('DQN')

./experiments/kuhn_poker_dqn_result/performance.csv


In [18]:
# Save model
agent.save('models/kuhn_poker_dqn/')

In [22]:
agent.weight_updates

812

In [19]:
eval_env_dqn_dqn = rlcard.make('kuhn-poker', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)

[-0.0012, 0.0012]


In [20]:
eval_env_dqn_dqn = rlcard.make('kuhn-poker', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, random_agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)

[0.2796, -0.2796]


In [21]:
eval_env_dqn_dqn = rlcard.make('kuhn-poker', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, cfr_agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)

[-0.0091, 0.0091]
